# Sonderübung 2

## Imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from gurobipy import *

### Achtung: 
die untenstehenden Befehle funktionieren nur mit einer aktuellen pandas Version: falls beim pandas.drop() Befehl, ein Fehler auftritt , dann schauen, dass pandas geupdated wird zb über "pip install pandas --upgrade" (Version 1.4.2. funktioniert), danach das Notebook komplett neustarten und nicht nur die Importzelle neu ausführen

## Parameter für das Optimierungsproblem

In [2]:
shares = ['AAPL', 'MSFT', 'XOM', 'JNJ', 'GE', 'BRK-B', 'FB', 
          'AMZN', 'WFC','T','GOOGL','PG','GOOG','VZ','PFE',
          'CVX','KO','HD','DIS','BAC','MRK','V','PM','CMCSA',
          'INTC','PEP','CSCO','C','GILD','IBM']

In [3]:
historical_prices = pd.read_excel('historical_prices.xlsx', index_col=0)
historical_returns = historical_prices / historical_prices.shift(1) - 1
historical_returns = historical_returns.drop(['2015-05-13'])
mu = historical_returns.mean()*252 # Estimation of mu
sigma = historical_returns.cov()*252 # Estimation of covariance-matrix

In [4]:
sigma.iloc[0:8,0:8]

,AAPL,MSFT,XOM,JNJ,GE,BRK-B,FB,AMZN
AAPL,0.077915,0.045575,0.031541,0.020971,0.028955,0.027391,0.040927,0.040994
MSFT,0.045575,0.072776,0.027717,0.025104,0.032222,0.029367,0.048484,0.051320
XOM,0.031541,0.027717,0.059183,0.022321,0.028922,0.027715,0.024145,0.024970
JNJ,0.020971,0.025104,0.022321,0.026636,0.019970,0.021105,0.022804,0.023365
GE,0.028955,0.032222,0.028922,0.019970,0.043824,0.026067,0.029822,0.028163
BRK-B,0.027391,0.029367,0.027715,0.021105,0.026067,0.031443,0.026365,0.025164
FB,0.040927,0.048484,0.024145,0.022804,0.029822,0.026365,0.106992,0.075159
AMZN,0.040994,0.051320,0.024970,0.023365,0.028163,0.025164,0.075159,0.121713


In [5]:
Sigma = sigma.values
Mu = mu.values
lamb = 10
n = Mu.size

## Aufgabe 2 b) 
Zeigen Sie, dass die Zielfunktion von $P$ mit den gegebenen Daten $\Sigma, \mu$ und $\lambda$ gleichmäßig konvex ist.

In [20]:
from numpy import linalg as LA
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

import random

Nach 2.5.10 Satz (C2-Charakterisierung von gleichmäßiger Konvexität).
Wir brechen die Eigenwerte von $D^{2}(f(x)) = \lambda\Sigma$ und zeigen der kleinste Eigenwert größer null ist. 

In [7]:
def test_gleichmaeßig_konvex(matrix):
    w,v = LA.eig(matrix)
    if (np.min(w) > 0): 
        print (f"gleichmäßig konvex with the smallest eigenvalue")
        return np.min(w)
    else:
        print (f" not gleichmäßig konvex")
D_2_fx = lamb * Sigma
test_gleichmaeßig_konvex(D_2_fx)

gleichmäßig konvex with the smallest eigenvalue


0.00778837301743193

## 2 c) Frank Wolfe
Implementieren Sie das Verfahren aus dem Algorithmus auf dem Aufgabenblatt. Geben Sie am Ende aus, wie viel in welche Aktien investiert wird. Beachten Sie die Hinweise auf dem Aufgabenblatt.

In [11]:
eps = 1e-4

In [12]:
def get_element_inM(n):
    a = np.random.rand(n,1)
    x_0 = (a/a.sum(axis=0,keepdims=1)).flatten()
    return x_0

In [13]:
Sigma = sigma.values
Mu = mu.values
lamb = 10
n = Mu.size
def func_min(x):
    return np.dot(-Mu.transpose(), x) + 0.5*lamb* np.dot(x.transpose(),np.dot(Sigma,x)) 
def gradien_func_min(x):
    return -Mu + lamb* np.dot(Sigma,x)

In [39]:
def problem_S(x_k,d_k):
    ''' return: t^k 
    '''
    fun = lambda t: func_min(x_k + t*d_k)
    bounds = Bounds(0+eps, 1- eps)
    t0 = random.uniform(0, 1)
    res = minimize(fun,t0, method='Nelder-Mead',bounds= bounds, tol=eps)
    return res.x.flatten()[0]

0.4342403679069809


In [16]:
x_1

array([0.03844208, 0.00187911, 0.02065977, 0.0018829 , 0.04606773,
       0.07877305, 0.01916301, 0.03387065, 0.01277654, 0.03875895,
       0.04698782, 0.04868994, 0.00953328, 0.01909641, 0.08091974,
       0.00633787, 0.07741274, 0.02192305, 0.02021178, 0.03729333,
       0.04129264, 0.02848419, 0.0280909 , 0.01777133, 0.05223196,
       0.03295293, 0.05254997, 0.02831307, 0.0444411 , 0.01319217])

In [58]:
def problem_Q(xk):
    '''return minpunkt y^k 
              minwert v^k
    '''
    fun = lambda x: np.dot(gradien_func_min(xk),(x-xk))
    con = lambda x: np.abs(np.sum(x) -1) 
    bounds = Bounds(np.zeros(n),np.ones(n)*np.inf)
    
    nlc = NonlinearConstraint(con, -eps, eps)
    res = minimize(fun,xk, method='L-BFGS-B',constraints=nlc,bounds=Bounds, tol=eps)
    return (res.fun, res.x)

In [59]:
res_q = problem_Q(x_0)

TypeError: zip() argument after * must be an iterable, not type

array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
       inf, inf, inf, inf])

In [44]:
x_1 = get_element_inM(n)
x_0 = get_element_inM(n)
d_1 = x_1 - x_0
print(problem_S(x_0, d_1))

0.9999


## 2 d) Python Lösung
Lösen Sie $P$ direkt mit Python und vergleichen Sie ihren Optimalpunkt mit dem aus Teil c). Ist dieses Ergebnis zu erwarten? Verwenden Sie Aufgabenteil b).

### Vergleich